In [1]:
import pandas as pd

csv_path = "flight_status_raw_20230430_20251213.csv"

df = pd.read_csv(csv_path)

print("✅ CSV 로드 완료")
print("행 수:", len(df))
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'flight_status_raw_20230430_20251213.csv'

In [ ]:
df["AIRLINE_KOREAN"].unique()

In [ ]:
df["AIRLINE_KOREAN"].value_counts()


In [ ]:
df["RMK_KOR"].value_counts()

In [ ]:
#타겟 변수(is_delay) 생성
def make_delay_label(x):
    if pd.isna(x):
        return 0
    if any(k in x for k in ["지연", "결항", "회항"]):
        return 1
    return 0

df["is_delay"] = df["RMK_KOR"].apply(make_delay_label)


In [ ]:
df["is_delay"].value_counts()

In [ ]:
import sys
!{sys.executable} -m pip install xlsxwriter


In [ ]:
# =====================================================
# ✈️ 항공사별 비행 횟수
# =====================================================
airline_counts = (
    df["AIRLINE_KOREAN"]
    .value_counts(dropna=False)
    .reset_index()
)
airline_counts.columns = ["AIRLINE_KOREAN", "total_flights"]

print("\n✈️ 항공사별 비행 횟수")
print(airline_counts)


In [ ]:
import pandas as pd

# =====================================================
# 2️⃣ BOARDING_KOR 전처리 (국내 출발지만 + 표준화)
# =====================================================

allowed_boarding_raw = [
    "제주",
    "서울/김포",
    "부산/김해",
    "청주",
    "대구",
    "광주",
    "인천",
    "여수",
    "울산",
    "포항/포항경주",
    "진주/사천",
    "군산",
    "원주",
    "무안",
    "양양"
]

boarding_replace_map = {
    "서울/김포": "김포",
    "부산/김해": "김해",
    "포항/포항경주": "포항경주",
    "진주/사천": "사천"
}

before_rows = len(df)
df = df[df["BOARDING_KOR"].isin(allowed_boarding_raw)].copy()
after_rows = len(df)

print(f"✂️ 출발지 필터링: {before_rows} → {after_rows}")

df["BOARDING_KOR"] = df["BOARDING_KOR"].replace(boarding_replace_map)

print("✅ BOARDING_KOR 전처리 완료")
print(df["BOARDING_KOR"].value_counts())

# =====================================================
# 3️⃣ 타겟 변수 생성 (지연 여부)
# =====================================================
def make_delay_label(x):
    if pd.isna(x):
        return 0
    if any(k in x for k in ["지연", "결항", "회항"]):
        return 1
    return 0

df["is_delay"] = df["RMK_KOR"].apply(make_delay_label)

print("\n📌 지연 분포")
print(df["is_delay"].value_counts())

# =====================================================
# 4️⃣ 날짜 파생
# =====================================================
df["flight_date"] = pd.to_datetime(df["FLIGHT_DATE"], format="%Y%m%d")

# =====================================================
# 5️⃣ 출발지 / 도착지 단순 카운트
# =====================================================
boarding_counts = (
    df["BOARDING_KOR"]
    .value_counts(dropna=False)
    .reset_index()
)
boarding_counts.columns = ["BOARDING_KOR", "total_flights"]

arrived_counts = (
    df["ARRIVED_KOR"]
    .value_counts(dropna=False)
    .reset_index()
)
arrived_counts.columns = ["ARRIVED_KOR", "total_flights"]

# =====================================================
# 🔹 5-1️⃣ 항공사별 비행 횟수 (신규)
# =====================================================
airline_counts = (
    df["AIRLINE_KOREAN"]
    .value_counts(dropna=False)
    .reset_index()
)
airline_counts.columns = ["AIRLINE_KOREAN", "total_flights"]

# =====================================================
# ✂️ 도착지 비행 횟수 24회 이상만 필터링
# =====================================================
arrived_counts_24plus = arrived_counts[arrived_counts["total_flights"] >= 24].copy()

print("\n📍 도착지 비행 횟수 24회 이상")
print(arrived_counts_24plus)

# =====================================================
# 6️⃣ 출발지 기준 지연 요약
# =====================================================
boarding_summary = (
    df
    .groupby("BOARDING_KOR")["is_delay"]
    .agg(
        total_flights="count",
        delay_count="sum",
        delay_rate="mean"
    )
    .reset_index()
    .sort_values("delay_count", ascending=False)
)

# =====================================================
# 7️⃣ 도착지 기준 지연 요약
# =====================================================
arrived_summary = (
    df
    .groupby("ARRIVED_KOR")["is_delay"]
    .agg(
        total_flights="count",
        delay_count="sum",
        delay_rate="mean"
    )
    .reset_index()
    .sort_values("delay_count", ascending=False)
)

# =====================================================
# 8️⃣ 엑셀 저장 (전처리된 결과만)
# =====================================================
save_path = "flight_airport_summary.xlsx"

with pd.ExcelWriter(save_path, engine="openpyxl") as writer:
    boarding_counts.to_excel(writer, sheet_name="boarding_counts", index=False)
    arrived_counts.to_excel(writer, sheet_name="arrived_counts", index=False)
    
        # ✅ 신규 추가
    arrived_counts_24plus.to_excel(
        writer,
        sheet_name="arrived_counts_24plus",
        index=False
    )
    
    airline_counts.to_excel(writer, sheet_name="airline_counts", index=False)  # 🔹 추가
    boarding_summary.to_excel(writer, sheet_name="boarding_summary", index=False)
    arrived_summary.to_excel(writer, sheet_name="arrived_summary", index=False)

print(f"\n✅ 엑셀 저장 완료: {save_path}")


In [ ]:
# =====================================================
# ✅ 도착지 24회 이상 리스트
# =====================================================
valid_arrived = arrived_counts_24plus["ARRIVED_KOR"].tolist()

print("✅ 허용 도착지 개수:", len(valid_arrived))
print(valid_arrived)


In [ ]:
# =====================================================
# ✅ 최종 분석용 데이터 필터링
# =====================================================
before_rows = len(df)

df_final = df[
    (df["BOARDING_KOR"].isin(df["BOARDING_KOR"].unique())) &  # 이미 국내만 남아 있음
    (df["ARRIVED_KOR"].isin(valid_arrived))
].copy()

after_rows = len(df_final)

print(f"📊 최종 데이터 필터링: {before_rows} → {after_rows}")


In [ ]:
print("\n✈️ 최종 출발지 분포")
print(df_final["BOARDING_KOR"].value_counts())

print("\n📍 최종 도착지 분포")
print(df_final["ARRIVED_KOR"].value_counts())

print("\n📌 최종 지연 분포")
print(df_final["is_delay"].value_counts())


In [ ]:
df_final.to_csv(
    "flight_status_final_filtered.csv",
    index=False,
    encoding="utf-8-sig"
)
print("✅ 최종 분석용 CSV 저장 완료")


# 최종 분석용 CSV 파일 

CSV 전체
 ↓
BOARDING_KOR ∈ allowed_boarding_raw   ← 여기서 국내만 남김 (핵심)
 ↓
슬래시 표준화 (김포, 김해, 사천…)
 ↓
도착지 24회 이상 필터
 ↓
최종 분석용 df


In [ ]:
import pandas as pd

df_weather = pd.read_csv(
    "airport_final.csv",
    encoding="euc-kr"
)


print("✅ 날씨 CSV 로드 완료")
print("행 수:", len(df_weather))
print("컬럼 수:", df_weather.shape[1])

df_weather.head()


In [ ]:
df_weather.info()


In [ ]:
missing_rate = (
    df_weather.isna().mean()
    .sort_values(ascending=False)
    .reset_index()
)
missing_rate.columns = ["column", "missing_rate"]

missing_rate


In [ ]:
numeric_cols = df_weather.select_dtypes(include=["int64", "float64"]).columns.tolist()

print("📊 수치형 컬럼 개수:", len(numeric_cols))
numeric_cols


In [ ]:
object_cols = df_weather.select_dtypes(include="object").columns.tolist()

object_cols

In [ ]:
# 랜덤 샘플 100개 추출
df_weather_sample = df_weather.sample(n=100, random_state=42)

# 엑셀 저장
sample_path = "airport_final_sample_100.xlsx"
df_weather_sample.to_excel(sample_path, index=False)

print(f"✅ 샘플 100개 엑셀 저장 완료: {sample_path}")


In [ ]:
station_list = (
    df_weather["지점명"]
    .dropna()
    .sort_values()
    .unique()
)

print("✅ 지점명 개수:", len(station_list))
station_list


In [ ]:
# 출발지 목록 추출
boarding_list = (
    df["BOARDING_KOR"]
    .dropna()
    .sort_values()
    .unique()
)

print("✅ BOARDING_KOR 개수:", len(boarding_list))
boarding_list


In [ ]:
# weather 지점명 → BOARDING_KOR 기준으로 변환
weather_to_boarding_map = {
    "광주": "광주",
    "대구" : "대구",
    "군산": "군산",
    "김포공항": "김포",
    "김해시": "김해",
    "무안공항": "무안",
    "양양공항": "양양",
    "여수공항": "여수",
    "울산공항": "울산",
    "원주": "원주",
    "인천공항": "인천",
    "제주공항": "제주",
    "청주": "청주",
    "포항": "포항경주",   # ⚠️ 표기 선택
    "진주": "사천"        # ⚠️ 네가 합의한 해석
}


In [ ]:
# ✅ 지점명 → BOARDING_KOR 생성 (핵심)
df_weather["BOARDING_KOR"] = df_weather["지점명"].map(weather_to_boarding_map)


In [ ]:
df["STD"].isna().sum()

In [ ]:
# =====================================================
# STD + FLIGHT_DATE → flight_datetime 생성
# =====================================================

# 1️⃣ STD 컬럼 상태 확인
print("STD 컬럼 정보")
df_final["STD"].info()
print("STD 결측 개수:", df["STD"].isna().sum())

# 2️⃣ STD 정규화 (HHMM → 4자리 문자열)
df_final["STD_clean"] = (
    df_final["STD"]
    .fillna(0)
    .astype(int)
    .astype(str)
    .str.zfill(4)
)

# 3️⃣ flight_datetime 생성 (STD 기준)
df_final["flight_datetime"] = pd.to_datetime(
    df_final["FLIGHT_DATE"].astype(str) + df_final["STD_clean"],
    format="%Y%m%d%H%M",
    errors="coerce"
)

# 4️⃣ 생성 결과 검증
print("\nNaT 개수:", df_final["flight_datetime"].isna().sum())

print("\n샘플 확인")
df_final[
    ["FLIGHT_DATE", "STD", "STD_clean", "flight_datetime"]
].sample(10, random_state=42)

In [ ]:
df_weather["일시"].value_counts().head(10)

In [ ]:
# =====================================================
# weather 일시 → weather_datetime 생성 (필수)
# =====================================================
df_weather["weather_datetime"] = pd.to_datetime(
    df_weather["일시"],
    errors="coerce"
)

In [ ]:
# =====================================================
# 1️⃣ 정렬 (필수)
# =====================================================
df_final = df_final.sort_values("flight_datetime")
df_weather = df_weather.sort_values("weather_datetime")

# =====================================================
# 2️⃣ merge_asof
# =====================================================
df_merged = pd.merge_asof(
    df_final,
    df_weather,
    left_on="flight_datetime",
    right_on="weather_datetime",
    left_by="BOARDING_KOR",
    right_by="BOARDING_KOR",
    direction="backward"
)


In [ ]:
print("전체 flight 행 수:", len(df_merged))
print(
    "날씨 붙은 행 수:",
    df_merged["weather_datetime"].notna().sum()
)
print(
    "날씨 미부착 행 수:",
    df_merged["weather_datetime"].isna().sum()
)
print(
    "날씨 부착 비율:",
    df_merged["weather_datetime"].notna().mean()
)


In [ ]:
(
    df_merged
    .groupby("BOARDING_KOR")["weather_datetime"]
    .apply(lambda x: x.notna().mean())
    .sort_values()
)


In [ ]:
save_path = "flight_weather_merged.csv"

df_merged.to_csv(
    save_path,
    index=False,
    encoding="utf-8-sig"
)

print(f"✅ CSV 저장 완료: {save_path}")
